In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import glob
from PIL import Image
import imagehash

In [ ]:
flower_label = {'rose' : 0,             # 장미
                'sunflower': 1,         # 해바라기
                'tulip' : 2,            # 튤립
                'carnation' : 3,        # 카네이션
                'gerbera' : 4,          # 거베라
                'lily' : 5,             # 백합
                'lisianthus' : 6,       # 리시안셔스
                'small_chrysanthemum' : 7,    # 소국
                'pompom_blooms' : 8,    # 퐁퐁국화
                'alstroemeria' : 9,     # 알스트로메리아
                'ranunculus' : 10,      # 라넌큘러스
                'freesia' : 11,         # 프리지아
                'iris' : 12,            # 붓꽃 (아이리스)
                'hydrangea' : 13,       # 수국
                'peony' : 14,            # 작약
                'stock' : 15,           # 스톡
                'spary_rose' : 16,      # 스프레이 장미
                'ranunculus_butterfly':17,  # 라넌큘러스 버터플라이
                }

In [ ]:
url = 'https://openapi.naver.com/v1/search/image'
header = {
    'X-Naver-Client-Id': 'iwPIgOh0e7a1fgeRVg3y',
    'X-Naver-Client-Secret' : 'rRjO7XI0eu'
}

# flowers = ['해바라기 한 송이', '거베라', '튤립 한 송이', '카네이션 한 송이', '리시안셔스 한 송이', '스프레이 장미 한 송이', '작약 한 송이', '라넌큘러스 한 송이', '프리지아 한 송이', '소국', '국화 한 송이', '백합 한 송이', '알스트로메리아']
# dirs = ['sunflower', 'gubera', 'tulip', 'canation', 'risiansus', 'spray_rose', 'jacyak', 'lanunculase', 'prizia', 'small_guka', 'guka', 'backhap', 'alstromeria']
flowers = ['장미 한 송이', '장미 생화']
dirs = ['rose', 'rose']

for i in range(len(flowers)):
    
    q = flowers[i]
    directory = dirs[i]
    
    if not os.path.isdir(f'./0_flower_data/{directory}'):
        os.makedirs(f'./0_flower_data/{directory}')
    
    for s in [1, 101, 201, 301, 401, 501]:
        print(f'{q} : {s}번째')
        params = {
            'query' : q,
            'display' : 100,
            'start' : s
        }

        req = requests.get(url=url, params=params, headers=header).json()

        items = req['items']

        cnt = int(os.listdir(f'./0_flower_data/{directory}/')[-1][:-4][-4:])+1 if os.listdir(f'./0_flower_data/{directory}/') else 0

        for item in items:
            try:
                img_url = item['link']
                response = requests.get(img_url)
                fn = str(cnt).rjust(4,'0')
                with open(f'./0_flower_data/{directory}/{directory}_{fn}.jpg', 'wb') as f:
                    f.write(response.content)
            except:
                print('여긴 안 되네양')
            cnt += 1

In [ ]:
flower = 'rose'

hash_dict = {}
x = []
for f in os.listdir(f'./0_flower_data/{flower}'):

    image_files = glob.glob(f"./0_flower_data/{flower}/{f}")  # 이미지 파일 경로 수정

    for image_path in image_files:
        try:
            with Image.open(image_path) as img:
                img_hash = imagehash.phash(img)
                if img_hash not in hash_dict:
                    hash_dict[img_hash] = image_path
                else:
                    print(f"중복 이미지 발견: {image_path}")
                    x.append(image_path)
        except:
            x.append(image_path)

for i in x:
    print('이미지 삭제 : ', i)
    os.remove(i)

### RGB로 변경 및 ICC 프로파일 제거

In [ ]:
from PIL import Image

flower = 'rose'

for f in os.listdir(f'./0_flower_data/{flower}'):
    img_path = f'./0_flower_data/{flower}/{f}'
    with Image.open(img_path) as img:
        img = img.convert('RGB')
        img.save(img_path, icc_profile=None)  # ICC 프로파일 제거

### 이미지 파일 부재시, 라벨 파일도 함께 삭제

In [ ]:
for CLS in ['train', 'test']:
    img_list = os.listdir(f'./5_total_data/{CLS}/images')
    for f in os.listdir(f'./5_total_data/{CLS}/labels/'):
        f_jpg = f[:-4]+'.jpg'
        if f_jpg not in img_list:
            print(f)